# Lab #6

__TODO:__
1. Постройте список уникальных типов самолетов зарегистрированных в России
2. Какой тип самолета имеет самую раннюю дату выдачи сертификата?
3. Постройте запрос: Владелец аэропорта, Аэропорт, Пассажиропоток суммарный за 2018 год, Грузопоток суммарный за 2018 год
4. Перечислите аэропорты, где пассажиропоток меньше медианы, а  грузопоток больше медианы
5. Перечислите авиакомпании у которых нет типов воздушных судов зарегистрированных в России
6. Выведите список: Месяц, суммарный пассажиропоток за данный месяц, аэропорт в котором пассажиропоток в данном месяце максимальный
7. Выведите список: Тип аэропорта, средний грузопоток в месяц в аэропортах данного типа

__NOTE:__ Везде где не указан конкретный временной промежуток или другим образом не указано время был взят 2018 год (последний год достпуный в даных)

In [240]:
import pandas as pd
from itertools import chain

In [241]:
aircraft_data = pd.read_csv('https://raw.githubusercontent.com/sudakov/lab_it/master/aircraft.csv',
                            sep=';', date_parser=[-1])

In [242]:
airlines_data = pd.read_csv('https://raw.githubusercontent.com/sudakov/lab_it/master/airlines.csv', 
                            names=['Название_короткое', "Название_полное", "Аэропорт", "Флотилия"])

In [243]:
airports_data = pd.read_csv('https://raw.githubusercontent.com/sudakov/lab_it/master/airports.csv', 
                            names=['Аэропорт', 'Сертификат', "Владелец", "Тип"])

In [244]:
cargo_data = pd.read_csv('https://raw.githubusercontent.com/sudakov/lab_it/master/cargo%20transportation.csv', 
                         sep=';', na_values='***', dtype='str')

In [245]:
passenger_data = pd.read_csv('https://raw.githubusercontent.com/sudakov/lab_it/master/passenger%20'\
                             'transportation.csv', sep=';', na_values='***', dtype='str') 

---

## T1

Постройте список уникальных типов самолетов зарегистрированных в России

In [246]:
aircraft_data[(aircraft_data['Вид воздушного судна'] == 'самолет') &
              (aircraft_data['Опознавательные знаки'].str.startswith('RA'))]\
                ['Тип (наименование) воздушного судна'].unique()

array(['Птенец-2', 'С-2 Елочка', 'Небесный', ..., 'Ил-96-300ПУ(М1)',
       'Ил-96-400Т/IL96-400T', 'Ил-96-400 ВПУ'], dtype=object)

## T2

Какой тип самолета имеет самую раннюю дату выдачи сертификата?

In [247]:
aircraft_data[aircraft_data['дата действующего свидетельства о регистрации'] ==
              aircraft_data['дата действующего свидетельства о регистрации'].min()]\
                ['Тип (наименование) воздушного судна']

1107    Ан-2
Name: Тип (наименование) воздушного судна, dtype: object

## T3

Постройте запрос: Владелец аэропорта, Аэропорт, Пассажиропоток суммарный за 2018 год, Грузопоток суммарный за 2018 год

### airports_data preparation

In [248]:
airports_data['Наименование аэропорта РФ'] =\
    airports_data['Аэропорт'].astype('str').apply(lambda x: x.strip().split()[0])

### passenger_data preparation

In [249]:
passenger_data.drop(columns=['Январь - Декабрь'], inplace=True)
passenger_data.fillna(0, inplace=True)

In [250]:
for column in passenger_data.columns[1:]:
    passenger_data[column] = passenger_data[column].astype('str').apply(lambda x: int(x.replace(' ', '')))

In [251]:
passenger_data['Суммарный пасажиропоток'] = passenger_data.iloc[:, 2:].sum(axis=1).copy()

### cargo_data preparation

In [252]:
cargo_data['Год периода данных'] = cargo_data['Год периода данных'].apply(lambda x: int(x.replace(' ', '')))

In [253]:
cargo_data.drop(columns=['Январь - Декабрь'], inplace=True)
cargo_data.fillna(0, inplace=True)

In [254]:
for column in cargo_data.columns[2:]:
    cargo_data[column] = cargo_data[column].astype('str').apply(lambda x: float(x.replace(',', '.').replace(' ', '')))

In [255]:
cargo_data['Суммарный грузопоток'] = cargo_data.iloc[:, 2:].sum(axis=1).copy()

### merge

In [256]:
p1 = airports_data[['Владелец', 'Наименование аэропорта РФ']].\
    merge(passenger_data[passenger_data['Год периода данных'] == 2018], 
          on='Наименование аэропорта РФ', how='left').iloc[:, [0, 1, -1]].copy()

In [257]:
t3 = p1.merge(cargo_data[cargo_data['Год периода данных'] == 2018], 
         on='Наименование аэропорта РФ', how='left').iloc[:, [0, 1, 2, -1]].copy()

In [258]:
t3.drop_duplicates(keep='first', subset=['Наименование аэропорта РФ',
                                          "Суммарный грузопоток", "Суммарный пасажиропоток"], inplace=True)

## T4

Перечислите аэропорты, где пассажиропоток меньше медианы, а  грузопоток больше медианы

In [259]:
res[(res['Суммарный грузопоток'] > res['Суммарный грузопоток'].median())].head()

,Владелец,Наименование аэропорта РФ,Суммарный пасажиропоток,Суммарный грузопоток
46,ОАО «Аэропорт Мурманск»,Мурманск,713392.0,1287.50
50,ОАО «Комиавиатранс»,Сыктывкар,446783.0,469.08
51,ОАО «Комиавиатранс»,Воркута,17627.0,64.10
56,ОАО «Международный Аэропорт Иркутск»,Иркутск,1647187.0,6294.32
70,ОАО «Аэропорт Белоярский»,Белоярский,50291.0,63.50


In [260]:
res[(res['Суммарный пасажиропоток'] < res['Суммарный пасажиропоток'].median())]

,Владелец,Наименование аэропорта РФ,Суммарный пасажиропоток,Суммарный грузопоток


In [261]:
res[(res['Суммарный грузопоток'] > res['Суммарный грузопоток'].median()) 
    & (res['Суммарный пасажиропоток'] < res['Суммарный пасажиропоток'].median())]

,Владелец,Наименование аэропорта РФ,Суммарный пасажиропоток,Суммарный грузопоток


### T5 

Перечислите авиакомпании у которых нет типов воздушных судов зарегистрированных в России

In [262]:
aircraft_data.shape[0] == aircraft_data[(aircraft_data['Опознавательные знаки'].str.startswith('RA'))].shape[0]

True

In [263]:
def plane_checker(plane_string:str) -> bool:
    flag = True
    plane_list = [plane_name[:-4].strip() for plane_name in plane_string.split(',')]
    for plane in plane_list:
        if plane in aircraft_data['Тип (наименование) воздушного судна'].values:
            return False
        else:
            continue
    return True

In [264]:
airlines_data['Все самолеты не РФ'] =\
    airlines_data[airlines_data["Флотилия"].notna()]['Флотилия'].astype('str').apply(plane_checker)

In [265]:
airlines_data[airlines_data['Все самолеты не РФ'] == True]['Название_короткое'].values

array(['Авиа Менеджмент Груп', 'Аврора', 'Ай Флай', 'АТРАН',
       'Аэролимузин', 'Аэрофлот ОАО', 'ВИМ-АВИА', 'Джет Эйр Групп',
       'ДОНАВИА', 'Икар ООО', 'Катэкавиа', 'Когалымавиа ООО', 'МАЦ',
       'Меридиан', 'Нордавиа-регион.АЛ', 'Оренбургские АЛ ОАО', 'Победа',
       'Россия Авиакомпания ОАО', 'РОЯЛ ФЛАЙТ', 'Северный Ветер',
       'Сибирь', 'СИРИУС-АЭРО', 'Уральские авиалинии', 'ЭйрБриджКарго'],
      dtype=object)

## T6

Выведите список: Месяц, суммарный пассажиропоток за данный месяц, аэропорт в котором пассажиропоток в данном месяце максимальный

In [266]:
best_port = []
for column in passenger_data.columns[2:-1]:
    best_port.append(passenger_data.loc[(passenger_data['Год периода данных'] == 2018) &
                       (passenger_data[column] == passenger_data[column].max())]['Наименование аэропорта РФ'].values)

In [267]:
res = pd.DataFrame(passenger_data[passenger_data['Год периода данных'] == 2018].iloc[:,2:-1].sum(axis=0), 
             columns=['Cуммарный пассажиропоток']).copy()
res['Аэропорт'] = best_port
res

,Cуммарный пассажиропоток,Аэропорт
Январь,11975195,[Москва(Шереметьево)]
Февраль,10735375,[Москва(Шереметьево)]
Март,12395277,[Москва(Шереметьево)]
Апрель,13333555,[Москва(Шереметьево)]
Май,15059520,[Москва(Шереметьево)]
Июнь,18170319,[Москва(Шереметьево)]
Июль,20333167,[Москва(Шереметьево)]
Август,20693339,[Москва(Шереметьево)]
Сентябрь,18544868,[Москва(Шереметьево)]
Октябрь,0,[]


## T7

Выведите список: Тип аэропорта, средний грузопоток в месяц в аэропортах данного типа 

In [268]:
airports_data['Тип'] = airports_data['Тип'].str.strip().str.replace(' ', '')

In [269]:
port_dict = {}
for port_type in airports_data['Тип'].unique():
    port_dict[port_type] = airports_data[airports_data['Тип'] == port_type]['Наименование аэропорта РФ'].values

In [270]:
# Используем датафрейм из T3
# Суммарный грузопоток за год по все аэропортам --> 
# --> Средний грузопоток за год -(делим на кол-во месяцев)-> Средний грузопоток за месяц 

for key in port_dict: 
    port_dict[key] =t3[t3['Наименование аэропорта РФ'].isin(port_dict.get(key))]['Суммарный грузопоток'].mean()/12

In [271]:
pd.DataFrame.from_dict(port_dict, orient='index', columns=['Средний грузопоток в месяц'])

,Средний грузопоток в месяц
В,29.257500
Г,1.126694
Е,0.031667
А,41.462222
Д,0.089167
Б,120.467917
Б-1,NaN
В-1,NaN
А-2,14.908333
А-1,NaN
